In [31]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
import pyodbc
import datetime as dt


In [32]:
def clean_df(df):
    df['Route Date'] = pd.to_datetime(
        df['Route Date'], format="%Y-%m-%d", errors='coerce')
    df['Disposal Date'] = pd.to_datetime(
        df['Disposal Date'], format="%Y-%m-%d", errors='coerce')
    df['Tip In Time'] = pd.to_datetime(
        df['Tip In Time'], infer_datetime_format=True, errors='coerce')
    df['Tip Out Time'] = pd.to_datetime(
        df['Tip Out Time'], infer_datetime_format=True, errors='coerce')
    df['Tip In Time'] = df['Tip In Time'].fillna(pd.to_datetime('2021-10-20 00:00:00',infer_datetime_format=True))      
    df['Tip Out Time'] = df['Tip Out Time'].fillna(pd.to_datetime('2021-10-20 00:00:00',infer_datetime_format=True))    
    df['Tip In Time'] = df['Tip In Time'].dt.time
    df['Tip Out Time'] = df['Tip Out Time'].dt.time
    df['Tip In Time'] = pd.to_datetime(df['Tip In Time'], format='%H:%M:%S').apply(pd.Timestamp)
    df['Tip Out Time'] = pd.to_datetime(df['Tip Out Time'], format='%H:%M:%S').apply(pd.Timestamp)
    return df

In [33]:
PATH = "C:\\Users\\User\\Desktop\\blob_storage\\tip_we\\tip_all.xml"
ns = "{urn:schemas-microsoft-com:office:spreadsheet}"
etree = et.parse(PATH)
eroot = etree.getroot()
rows_attr = f"{ns}Row"
data_attr = f"{ns}Data"
data = list()

for r in eroot.iter(tag=rows_attr):
    row = np.array([])

    for d in r.iter(tag=data_attr):
        row = np.append(row, d.text)

    row = np.array(row)
    data.append(row)

df = pd.DataFrame(data=data[1:], columns=data[0]).iloc[1:]

In [56]:
df_1 = clean_df(df)

In [55]:
# df_1[df_1['Disposal Date'].eq()]
df.dropna(subset=['name', 'toy'])
df_1.dropna(subset=['Route Date', 'Disposal Date'])

,Sequence No,Booking No,Customer Details,Route No,Truck No,Route Date,Disposal Date,Tip Site,Tip In Time,Tip Out Time,...,Total Cost,Charge Rate,Total Charge,Waste Type,Weight,UOM,Docket No,Gross Weight,Tare Weight,Branch
1,1,0.0,None,RED-2,XVA350,2016-11-01,2016-11-02,DOYLES Fairfield,1900-01-01 00:00:00,1900-01-01 04:03:59,...,0.0,0.0,0.0,PC,3.41,TONNES,NOT ISSUED,0,0,None
2,1,0.0,None,RLE-2,CG40UW,2016-11-01,2016-11-02,VEOLIA Clyde,1900-01-01 00:00:00,1900-01-01 02:30:49,...,0.0,0.0,0.0,GW,5.22,Tonnes,AU230144699,0,0,None
3,1,0.0,None,RED-3,XVA350,2016-11-02,2016-11-03,DOYLES Fairfield,1900-01-01 00:00:00,1900-01-01 04:29:29,...,0.0,0.0,0.0,PC,3.8,TONNES,None,0,0,None
4,1,0.0,None,RL5-3,CH32FV,2016-11-02,2016-11-02,DOYLES Fairfield,1900-01-01 00:00:00,1900-01-01 14:42:39,...,0.0,0.0,0.0,PC,1.62,TONNES,49112,0,0,None
5,1,0.0,None,RED-4,XVA350,2016-11-03,2016-11-04,DOYLES Fairfield,1900-01-01 00:00:00,1900-01-01 04:34:03,...,0.0,0.0,0.0,PC,3.46,TONNES,None,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86830,291264532,0.0,None,RLD-5,CL02TB,2021-10-15,2021-10-15,Suez - Wetherill Park,1900-01-01 00:00:00,1900-01-01 12:41:13,...,7.32,1.0,0.0,GW,7.32,TONNES,DR150086372,20.58,13.26,None
86831,291451347,0.0,None,BR3-5,XN06OJ,2021-10-15,2021-10-15,Suez - Wetherill Park,1900-01-01 00:00:00,1900-01-01 14:31:38,...,5.76,1.0,0.0,GW,5.76,TONNES,DR150086412,20.6,14.84,None
86832,291451354,0.0,None,RLH-5,XN19IN,2021-10-15,2021-10-15,Suez - Wetherill Park,1900-01-01 00:00:00,1900-01-01 14:32:10,...,1.92,1.0,0.0,GW,1.92,TONNES,DR150086413,9.84,7.92,None
86833,291264484,0.0,None,RLK-5,XN71FD,2021-10-15,2021-10-15,Suez - Seven Hills,1900-01-01 11:06:00,1900-01-01 11:21:00,...,0.12,0.0,0.0,GW,.12,TONNES,SH230212200.0,5.32,5.2,None


In [ ]:
server = 'localhost'
database = 'STAGE'
username = 'SA'
password = 'Sydwaste123#'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server +
                      ';DATABASE='+database+';UID='+username+';PWD=' + password)

cursor = cnxn.cursor()

for index, row in df_1.iterrows():
    print(index)
    cursor.execute("INSERT INTO [experiment].[WE_TIPPING_SCH_1].[WE_TIPPING_TB_1] \
    (Sequence_No,Booking_No,Customer_Details,Route_No,Truck_No,\
    Route_Date,Disposal_Date,Tip_Site,Tip_In_Time,\
    Tip_Out_Time, Cost_Rate, Total_Cost, Charge_Rate,\
    Total_Charge, Waste_Type, Weight, UOM, Docket_No,\
    Gross_Weight, Tare_Weight, Branch) \
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                   row['Sequence No'],
                   row['Booking No'],
                   row['Customer Details'],
                   row['Route No'],
                   row['Truck No'],
                   row['Route Date'],
                   row['Disposal Date'],
                   row['Tip Site'],
                   row['Tip In Time'],
                   row['Tip Out Time'],
                   row['Cost Rate'],
                   row['Total Cost'],
                   row['Charge Rate'],
                   row['Total Charge'],
                   row['Waste Type'],
                   row['Weight'],
                   row['UOM'],
                   row['Docket No'],
                   row['Gross Weight'],
                   row['Tare Weight'],
                   row['Branch'])
cnxn.commit()
cursor.close()
cnxn.close()
